# Sequential Chains
Used to solve sequential problems. Requires more than one user input.

In [ ]:
destination_prompt = PromptTemplate(
    input_variables=['destination'],
    template='I am planning a trip to {destination}. Can you suggest some activities I can do there?'
)

activities_prompt = PromptTemplate(
    input_variables=['activities'],
    template='I only have one day, so can you create an itinerary from your top three activities: {activities}?'
)

llm = ChatOpenAI(model='gpt-4o-mini', api_key=api_key)

seq_chain = ({"activities": destination_prompt | llm | StrOutputParser()}
             | activities_prompt | llm | StrOutputParser())

print(seq_chain.invoke({"destination": "Paris"}))

### LangChain Agents
**Agents**: use LLMs to take actions
**Tools**: functions called by the agent

#### ReAct agents
Reason + Act

We use **LangGraph** which is a branch of LangChain centered around designing agent systems. I provides a unified, tool-agnostic syntax

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_community.agent_toolkits.load_tools import load_tools


llm = ChatOpenAI(model='gpt-4o-mini', api_key=api_key)
tools = load_tools(['llm-math'], llm=llm)
agent = create_react_agent(llm, tools=tools)

messages = agent.invoke({'message': [('human', 'What is the square root of 101?'),]})

print(messages['messages'][-1]['content'])

#### Custom tools for agents

In [ ]:
from langchain_core.tools import tool

@tool
def financial_report(company_name: str, revenue: float, expenses: float) -> str:
    """
    Generate a financial report for a company.
    """
    profit = revenue - expenses
    return f"Company: {company_name}\nRevenue: ${revenue}\nExpenses: ${expenses}\nProfit: ${profit}"

print(financial_report.name)
print(financial_report.description)
print(financial_report.model_json_schema)
print(financial_report.return_direct)
print(financial_report.args)

agent = create_react_agent(llm, tools=[financial_report])

messages = agent.invoke({'message': [('human', 'TechStack generated  made $10 million in revenue and $7 million in expenses. Can you generate a financial report for them?'),]})


financial_report
Generate a financial report for a company.
<bound method BaseModel.model_json_schema of <class 'langchain_core.tools.structured.StructuredTool'>>
False
{'company_name': {'title': 'Company Name', 'type': 'string'}, 'revenue': {'title': 'Revenue', 'type': 'number'}, 'expenses': {'title': 'Expenses', 'type': 'number'}}
